In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler
import random
import xgboost as xgb
from sklearn.model_selection import train_test_split

# Setting a random seed for reproducibility
np.random.seed(42)

###################### -- HELPER FUNCTIONS -- ######################
def merge_predictions(preds_a, preds_b):
    merged_predictions = []
    for i in range(len(preds_a)):
        
        # Replacing 9 back to 24 if needed
        decoded_a = preds_a[i] if preds_a[i] != 9 else 24
        decoded_b = preds_b[i] if preds_b[i] != 9 else 24

        ascii_a = decoded_a + 65
        ascii_b = decoded_b + 65

        sum_pred = normalize_ascii_sum(ascii_a + ascii_b)
        merged_predictions.append((i, chr(sum_pred)))
    return merged_predictions

def separate_test_sets(file_path):
    test_data = pd.read_csv(file_path)
    test_a_columns = [col for col in test_data.columns if 'pixel_a' in col]
    test_b_columns = [col for col in test_data.columns if 'pixel_b' in col]
    test_a = test_data[test_a_columns]
    test_b = test_data[test_b_columns]
    test_a.columns = [col.replace('_a', '') for col in test_a.columns]
    test_b.columns = [col.replace('_b', '') for col in test_b.columns]
    return test_a, test_b

def normalize_ascii_sum(ascii_sum):
    while ascii_sum > 122:  # 'z' is ASCII 122
        ascii_sum -= 65  # 122 ('z') - 65 ('A') + 1
    return int(ascii_sum)

def save_predictions_to_csv(filename, predictions):
    with open(filename, 'w') as file:
        file.write("id,label\n")
        for id, label in predictions:
            file.write(f"{id},{label}\n")

###################### -- Random Forest Custom  -- ######################

class RandomForestClassifier:
    def __init__(self, n_estimators=100, max_features=3, max_depth=10, min_samples_split=2):
        self.n_estimators = n_estimators
        self.max_features = max_features
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.trees = []

    @staticmethod
    def _information_gain(left_child, right_child):
        parent = left_child + right_child
        p_parent = parent.count(1) / len(parent) if len(parent) > 0 else 0
        p_left = left_child.count(1) / len(left_child) if len(left_child) > 0 else 0
        p_right = right_child.count(1) / len(right_child) if len(right_child) > 0 else 0
        IG_p = entropy(p_parent)
        IG_l = entropy(p_left)
        IG_r = entropy(p_right)
        return IG_p - len(left_child) / len(parent) * IG_l - len(right_child) / len(parent) * IG_r

    @staticmethod
    def _draw_bootstrap(X_train, y_train):
        bootstrap_indices = list(np.random.choice(range(len(X_train)), len(X_train), replace=True))
        oob_indices = [i for i in range(len(X_train)) if i not in bootstrap_indices]
        X_bootstrap = X_train.iloc[bootstrap_indices].values
        y_bootstrap = y_train[bootstrap_indices]
        X_oob = X_train.iloc[oob_indices].values
        y_oob = y_train[oob_indices]
        return X_bootstrap, y_bootstrap, X_oob, y_oob

    @staticmethod
    def _oob_score(tree, X_test, y_test):
        mis_label = 0
        for i in range(len(X_test)):
            pred = RandomForestClassifier._predict_tree(tree, X_test[i])
            if pred != y_test[i]:
                mis_label += 1
        return mis_label / len(X_test)

    @staticmethod
    def _find_split_point(X_bootstrap, y_bootstrap, max_features):
        feature_ls = list()
        num_features = len(X_bootstrap[0])

        while len(feature_ls) <= max_features:
            feature_idx = random.sample(range(num_features), 1)
            if feature_idx not in feature_ls:
                feature_ls.extend(feature_idx)

        best_info_gain = -999
        node = None
        for feature_idx in feature_ls:
            for split_point in X_bootstrap[:, feature_idx]:
                left_child = {'X_bootstrap': [], 'y_bootstrap': []}
                right_child = {'X_bootstrap': [], 'y_bootstrap': []}

                if type(split_point) in [int, float]:
                    for i, value in enumerate(X_bootstrap[:, feature_idx]):
                        if value <= split_point:
                            left_child['X_bootstrap'].append(X_bootstrap[i])
                            left_child['y_bootstrap'].append(y_bootstrap[i])
                        else:
                            right_child['X_bootstrap'].append(X_bootstrap[i])
                            right_child['y_bootstrap'].append(y_bootstrap[i])
                else:
                    for i, value in enumerate(X_bootstrap[:, feature_idx]):
                        if value == split_point:
                            left_child['X_bootstrap'].append(X_bootstrap[i])
                            left_child['y_bootstrap'].append(y_bootstrap[i])
                        else:
                            right_child['X_bootstrap'].append(X_bootstrap[i])
                            right_child['y_bootstrap'].append(y_bootstrap[i])

                split_info_gain = RandomForestClassifier._information_gain(left_child['y_bootstrap'], right_child['y_bootstrap'])
                if split_info_gain > best_info_gain:
                    best_info_gain = split_info_gain
                    left_child['X_bootstrap'] = np.array(left_child['X_bootstrap'])
                    right_child['X_bootstrap'] = np.array(right_child['X_bootstrap'])
                    node = {'information_gain': split_info_gain,
                            'left_child': left_child,
                            'right_child': right_child,
                            'split_point': split_point,
                            'feature_idx': feature_idx}
        return node

    @staticmethod
    def _terminal_node(node):
        y_bootstrap = node['y_bootstrap']
        return max(y_bootstrap, key=y_bootstrap.count)

    @staticmethod
    def _split_node(node, max_features, min_samples_split, max_depth, depth):
        left_child = node['left_child']
        right_child = node['right_child']

        if len(left_child['y_bootstrap']) == 0 or len(right_child['y_bootstrap']) == 0:
            empty_child = {'y_bootstrap': left_child['y_bootstrap'] + right_child['y_bootstrap']}
            node['left_split'] = RandomForestClassifier._terminal_node(empty_child)
            node['right_split'] = RandomForestClassifier._terminal_node(empty_child)
            return

        if depth >= max_depth:
            node['left_split'] = RandomForestClassifier._terminal_node(left_child)
            node['right_split'] = RandomForestClassifier._terminal_node(right_child)
            return

        if len(left_child['X_bootstrap']) <= min_samples_split:
            node['left_split'] = RandomForestClassifier._terminal_node(left_child)
        else:
            node['left_split'] = RandomForestClassifier._find_split_point(left_child['X_bootstrap'], left_child['y_bootstrap'], max_features)
            RandomForestClassifier._split_node(node['left_split'], max_features, min_samples_split, max_depth, depth + 1)

        if len(right_child['X_bootstrap']) <= min_samples_split:
            node['right_split'] = RandomForestClassifier._terminal_node(right_child)
        else:
            node['right_split'] = RandomForestClassifier._find_split_point(right_child['X_bootstrap'], right_child['y_bootstrap'], max_features)
            RandomForestClassifier._split_node(node['right_split'], max_features, min_samples_split, max_depth, depth + 1)

    @staticmethod
    def _build_tree(X_bootstrap, y_bootstrap, max_depth, min_samples_split, max_features):
        root_node = RandomForestClassifier._find_split_point(X_bootstrap, y_bootstrap, max_features)
        RandomForestClassifier._split_node(root_node, max_features, min_samples_split, max_depth, 1)
        return root_node

    def fit(self, X_train, y_train):
        for _ in range(self.n_estimators):
            X_bootstrap, y_bootstrap, _, _ = RandomForestClassifier._draw_bootstrap(X_train, y_train)
            tree = RandomForestClassifier._build_tree(X_bootstrap, y_bootstrap, self.max_depth, self.min_samples_split, self.max_features)
            self.trees.append(tree)

    @staticmethod
    def _predict_tree(tree, x):
        feature_idx = tree['feature_idx']
        if x[feature_idx] <= tree['split_point']:
            if type(tree['left_split']) == dict:
                return RandomForestClassifier._predict_tree(tree['left_split'], x)
            else:
                return tree['left_split']
        else:
            if type(tree['right_split']) == dict:
                return RandomForestClassifier._predict_tree(tree['right_split'], x)
            else:
                return tree['right_split']

    def predict(self, X_test):
        preds = [max(set(RandomForestClassifier._predict_tree(tree, x) for tree in self.trees), key=list.count) for x in X_test]
        return preds

def startRandomForest(X_train, y_train, X_val, y_val):
    # Définir les grilles de recherche pour les hyperparamètres
    n_estimators_options = [100, 200, 300]
    max_features_options = [3, 5, 7]
    max_depth_options = [10, 15, 20]
    min_samples_split_options = [2, 4, 6]

    best_forest = None
    best_accuracy = 0
    best_params = None

    for n_estimators in n_estimators_options:
        for max_features in max_features_options:
            for max_depth in max_depth_options:
                for min_samples_split in min_samples_split_options:
                    forest = RandomForestClassifier(n_estimators, max_features, max_depth, min_samples_split)
                    forest.fit(X_train, y_train)
                    # Évaluer sur l'ensemble de validation
                    predictions = forest.predict(X_val)
                    acc = sum(predictions == y_val) / len(y_val)
                    print(f"n_estimators: {n_estimators} ; max_features: {max_features} ; Depth: {max_depth} ; min_samples_split: {min_samples_split} ; Accuracy: {acc * 100:.2f}%")
                    if acc > best_accuracy:
                        best_accuracy = acc
                        best_forest = forest
                        best_params = (n_estimators, max_features, max_depth, min_samples_split)
                        print(f"New best: {best_params} // Accuracy : {best_accuracy}")

    print(f"Best model : {best_params} // Accuracy : {best_accuracy}")
    return best_forest


###################### -- CNN IMPLEMENTATION -- ######################

class CNNClassifier:
    def __init__(self, input_shape, num_classes):
        self.model = self._build_model(input_shape, num_classes)

    def _build_model(self, input_shape, num_classes):
        model = Sequential([
            Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same'),
            BatchNormalization(),
            Conv2D(32, (3, 3), activation='relu', padding='same'),
            BatchNormalization(),
            MaxPooling2D((2, 2)),
            Dropout(0.25),
            Conv2D(64, (3, 3), activation='relu', padding='same'),
            BatchNormalization(),
            Conv2D(64, (3, 3), activation='relu', padding='same'),
            BatchNormalization(),
            MaxPooling2D((2, 2)),
            Dropout(0.25),
            Flatten(),
            Dense(512, activation='relu'),
            BatchNormalization(),
            Dropout(0.5),
            Dense(num_classes, activation='softmax')
        ])

        optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
        model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
        return model

    def train(self, x_train, y_train, x_val, y_val, epochs=50, batch_size=64):
        datagen = ImageDataGenerator(
            rotation_range=10,
            zoom_range=0.1,
            width_shift_range=0.1,
            height_shift_range=0.1
        )
        datagen.fit(x_train)
        reduce_lr = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)
        self.model.fit(
            datagen.flow(x_train, y_train, batch_size=batch_size),
            epochs=epochs,
            validation_data=(x_val, y_val),
            callbacks=[reduce_lr]
        )

    def predict(self, x_test):
        return self.model.predict(x_test)

def startCNN(X, Y, input_shape, num_classes):
    X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.1, random_state=42)
    cnn = CNNClassifier(input_shape, num_classes)
    cnn.train(X_train, Y_train, X_val, Y_val)
    return cnn

###################### -- XGBOOST Implementation -- ######################

class XGBoostClassifier:
    def __init__(self, max_depth=7, eta=0.1, num_class=24):
        self.params = {
            'objective': 'multi:softmax',
            'num_class': num_class,
            'booster': 'gbtree',
            'eval_metric': 'merror',
            'eta': eta,
            'max_depth': max_depth,
        }

    def train(self, train_data, train_labels, validation_data, validation_labels):
        dtrain = xgb.DMatrix(train_data, label=train_labels)
        dval = xgb.DMatrix(validation_data, label=validation_labels)
        watchlist = [(dtrain, 'train'), (dval, 'validation')]
        self.model = xgb.train(self.params, dtrain, num_boost_round=200, evals=watchlist, early_stopping_rounds=20, verbose_eval=False)

    def compute_predictions(self, data):
        ddata = xgb.DMatrix(data)
        return self.model.predict(ddata)

    def compute_accuracy(self, preds, labels):
        return np.mean(preds == labels)

def startXGBOOST(train_data_normalized, train_labels, validation_data_normalized, validation_labels):
    learning_rates = [0.01, 0.05, 0.1, 0.2, 0.3]
    max_depths = [3, 4, 5, 6, 7, 8, 9, 10]
    learning_rates = [0.1]
    max_depths = [7]
    results_XGB = []
    best_acc_xgb = 0
    best_lr = None
    best_depth = None
    num_class = len(np.unique(train_labels))
    print(num_class)

    for lr in learning_rates:
        for depth in max_depths:
            xgb_model = XGBoostClassifier(max_depth=depth, eta=lr, num_class=num_class)
            xgb_model.train(train_data_normalized, train_labels, validation_data_normalized, validation_labels)
            val_preds = xgb_model.compute_predictions(validation_data_normalized)
            acc = xgb_model.compute_accuracy(val_preds, validation_labels)
            print(f"LR: {lr} ; Depth: {depth} ; Accuracy: {acc * 100:.2f}%")
            if acc > best_acc_xgb:
                best_acc_xgb = acc
                best_lr = lr
                best_depth = depth
                print("New best")
            results_XGB.append((lr, depth, acc))

    best_xgb_model = XGBoostClassifier(max_depth=best_depth, eta=best_lr, num_class=num_class)
    best_xgb_model.train(train_data_normalized, train_labels, validation_data_normalized, validation_labels)

    return best_xgb_model

###################### -- DATA HANDLING AND TRAINING -- ######################

# Load your dataset here
mnist_sign_train = pd.read_csv('sign_mnist_train.csv')
test_a, test_b = separate_test_sets('test.csv')

mnist_sign_train['label'] = mnist_sign_train['label'].replace(24, 9) ## Because

# Préparation des données de training pour XGBoost & CNN
features_xgb = mnist_sign_train.drop('label', axis=1).values / 255.0
labels_xgb = mnist_sign_train['label'].values
features_cnn = features_xgb.reshape((-1, 28, 28, 1))
labels_cnn = tf.keras.utils.to_categorical(labels_xgb, num_classes=25)

# Préparation des données de test pour XGBoost & CNN
normalized_test_a_cnn = test_a.values.reshape((-1, 28, 28, 1)) / 255.0
normalized_test_b_cnn = test_b.values.reshape((-1, 28, 28, 1)) / 255.0
normalized_test_a_xgb = test_a.values / 255.0
normalized_test_b_xgb = test_b.values / 255.0

#Split Training Validation pour XGB
split_index = int(0.8 * len(features_xgb))
x_train_xgb, x_val_xgb = features_xgb[:split_index], features_xgb[split_index:]
y_train_xgb, y_val_xgb = labels_xgb[:split_index], labels_xgb[split_index:]


In [62]:
# Train XGB Model
best_xgb_model = startXGBOOST(x_train_xgb, y_train_xgb, x_val_xgb, y_val_xgb)

LR: 0.01 ; Depth: 3 ; Accuracy: 77.13%
New best
LR: 0.01 ; Depth: 4 ; Accuracy: 87.91%
New best
LR: 0.01 ; Depth: 5 ; Accuracy: 92.73%
New best
LR: 0.01 ; Depth: 6 ; Accuracy: 95.16%
New best
LR: 0.01 ; Depth: 7 ; Accuracy: 96.30%
New best
LR: 0.01 ; Depth: 8 ; Accuracy: 96.78%
New best
LR: 0.01 ; Depth: 9 ; Accuracy: 97.32%
New best
LR: 0.01 ; Depth: 10 ; Accuracy: 97.47%
New best
LR: 0.05 ; Depth: 3 ; Accuracy: 95.81%
LR: 0.05 ; Depth: 4 ; Accuracy: 98.22%
New best
LR: 0.05 ; Depth: 5 ; Accuracy: 98.83%
New best
LR: 0.05 ; Depth: 6 ; Accuracy: 99.11%
New best
LR: 0.05 ; Depth: 7 ; Accuracy: 99.27%
New best
LR: 0.05 ; Depth: 8 ; Accuracy: 99.18%
LR: 0.05 ; Depth: 9 ; Accuracy: 99.25%
LR: 0.05 ; Depth: 10 ; Accuracy: 99.29%
New best
LR: 0.1 ; Depth: 3 ; Accuracy: 98.63%
LR: 0.1 ; Depth: 4 ; Accuracy: 99.27%
LR: 0.1 ; Depth: 5 ; Accuracy: 99.38%
New best
LR: 0.1 ; Depth: 6 ; Accuracy: 99.42%
New best
LR: 0.1 ; Depth: 7 ; Accuracy: 99.54%
New best
LR: 0.1 ; Depth: 8 ; Accuracy: 99.53%
LR

In [53]:
# Train CNN Model
cnn_model = startCNN(features_cnn, labels_cnn, input_shape=(28, 28, 1), num_classes=25)


Epoch 1/50
387/387 [==============================] - 39s 97ms/step - loss: 1.0202 - accuracy: 0.6962 - val_loss: 3.4025 - val_accuracy: 0.2145 - lr: 0.0010
Epoch 2/50
387/387 [==============================] - 36s 93ms/step - loss: 0.2316 - accuracy: 0.9254 - val_loss: 0.0188 - val_accuracy: 0.9975 - lr: 9.0000e-04
Epoch 3/50
387/387 [==============================] - 37s 94ms/step - loss: 0.1165 - accuracy: 0.9636 - val_loss: 0.2669 - val_accuracy: 0.8922 - lr: 8.1000e-04
Epoch 4/50
387/387 [==============================] - 36s 93ms/step - loss: 0.0828 - accuracy: 0.9739 - val_loss: 0.0025 - val_accuracy: 1.0000 - lr: 7.2900e-04
Epoch 5/50
387/387 [==============================] - 40s 103ms/step - loss: 0.0630 - accuracy: 0.9813 - val_loss: 0.0035 - val_accuracy: 1.0000 - lr: 6.5610e-04
Epoch 6/50
387/387 [==============================] - 38s 98ms/step - loss: 0.0501 - accuracy: 0.9853 - val_loss: 0.0013 - val_accuracy: 1.0000 - lr: 5.9049e-04
Epoch 7/50
387/387 [=================

In [8]:
# Train Random Forest Model
best_rf_model = startRandomForest(x_train_xgb, y_train_xgb, x_val_xgb, y_val_xgb)

AttributeError: 'numpy.ndarray' object has no attribute 'iloc'

In [63]:
# XGBoost Predictions
xgb_preds_a = best_xgb_model.compute_predictions(normalized_test_a_xgb)
xgb_preds_b = best_xgb_model.compute_predictions(normalized_test_b_xgb)
xgb_merged_predictions = merge_predictions(xgb_preds_a, xgb_preds_b)

In [54]:
# CNN Predictions
cnn_preds_a = cnn_model.predict(normalized_test_a_cnn)
cnn_preds_b = cnn_model.predict(normalized_test_b_cnn)
cnn_merged_predictions = merge_predictions(np.argmax(cnn_preds_a, axis=1), np.argmax(cnn_preds_b, axis=1))

94/94 [==============================] - 1s 8ms/step


In [ ]:
# Random Forest Predictions
rfc_preds_a = best_rfc_model.predict(normalized_test_a_xgb)
rfc_preds_b = best_rfc_model.predict(normalized_test_b_xgb)
rfc_merged_predictions = merge_predictions(rfc_preds_a, rfc_preds_b)

In [64]:
# Save XGB
save_predictions_to_csv("xgb_predictions.csv", xgb_merged_predictions)


In [55]:
# Save CNN
save_predictions_to_csv("cnn_predictions.csv", cnn_merged_predictions)


In [ ]:
# Save Random Forest
save_predictions_to_csv("rfc_predictions.csv", rfc_merged_predictions)